In [7]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

#### Преобразуем данные о жанрах в строки

In [3]:
movies.head(10)

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   
5        6                         Heat (1995)   
6        7                      Sabrina (1995)   
7        8                 Tom and Huck (1995)   
8        9                 Sudden Death (1995)   
9       10                    GoldenEye (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
5                        Action|Crime|Thriller  
6                               Comedy|Romance  
7                           Adventure|Children  
8                                       Action  
9                    Action|Adventure|Thriller

In [4]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [5]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [6]:
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

#### Преобразуем данные о жанрах в векторы и найдём ближайшие по жанрам к примеру

In [8]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)

In [9]:
neigh = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=7)

In [12]:
test = change_string("Adventure|Comedy|Fantasy|Crime")

X_tfidf_test = tfidf.transform([test])

result = neigh.kneighbors(X_tfidf_test, return_distance=True)
result

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608]]),
 array([[6774, 9096, 5636, 6723, 3376, 7496, 9717]]))

In [14]:
movies.iloc[result[1][0]]

movieId                                              title  \
6774    60074                                     Hancock (2008)   
9096   143559                                L.A. Slasher (2015)   
5636    27368  Asterix & Obelix: Mission Cleopatra (Astérix &...   
6723    58972                                Nim's Island (2008)   
3376     4591                             Erik the Viking (1989)   
7496    82854                          Gulliver's Travels (2010)   
9717   188833              The Man Who Killed Don Quixote (2018)   

                                     genres  
6774  Action|Adventure|Comedy|Crime|Fantasy  
9096                   Comedy|Crime|Fantasy  
5636               Adventure|Comedy|Fantasy  
6723               Adventure|Comedy|Fantasy  
3376               Adventure|Comedy|Fantasy  
7496               Adventure|Comedy|Fantasy  
9717               Adventure|Comedy|Fantasy

#### Добавим к данным о жанрах информацию о тегах

In [17]:
movies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [18]:
tags.head()

userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200

In [20]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')
movies_with_tags.head()

movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        2    Jumanji (1995)                   Adventure|Children|Fantasy   
1        2    Jumanji (1995)                   Adventure|Children|Fantasy   

   userId               tag     timestamp  
0   336.0             pixar  1.139046e+09  
0   474.0             pixar  1.137207e+09  
0   567.0               fun  1.525286e+09  
1    62.0           fantasy  1.528844e+09  
1    62.0  magic board game  1.528844e+09

#### Удалим теги-дубликаты и отсутствующие значения

In [21]:
movies_with_tags[movies_with_tags.title == 'Toy Story (1995)']

movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId    tag     timestamp  
0   336.0  pixar  1.139046e+09  
0   474.0  pixar  1.137207e+09  
0   567.0    fun  1.525286e+09

In [26]:
movies_with_tags.tag.unique()
movies_with_tags.dropna(inplace=True)

In [28]:
# Общее количество фильмов
movies_with_tags.title.unique().shape

(1572,)

#### Преобразуем данные тегов и векторизуем их

In [82]:
def change_string(s):
    return str(s).replace(' ', '').replace('-', '').lower()

tag_strings = []
movies = []

for movie, group in tqdm_notebook(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([change_string(s) for s in group.tag.values]))
    movies.append(movie)

<ipython-input-82-8817db38ff17>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie, group in tqdm_notebook(movies_with_tags.groupby('title')):


  0%|          | 0/1572 [00:00<?, ?it/s]

In [83]:
tag_strings[:5]

['artistic funny humorous inspiring intelligent quirky romance zooeydeschanel',
 'lawyers',
 'creepy suspense',
 'shakespearesortof',
 'dogs remake']

#### Найдём ближайших соседей

In [84]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(tag_strings)

In [85]:
neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='manhattan') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='manhattan', n_jobs=-1, n_neighbors=10)

In [97]:
test = 'highschool pixar'

X_tfidf_test = tfidf.transform([test])
result = neigh.kneighbors(X_tfidf_test, return_distance=True)
result

(array([[0.86728067, 1.13271933, 1.13271933, 1.13271933, 1.13271933,
         1.13271933, 1.13271933, 1.13271933, 1.13271933, 1.13271933]]),
 array([[ 211,  590,  439,  547,  546,  336, 1303,  432,  940,  951]]))

In [98]:
for i in result[1][0]:
    print(movies[i], tag_strings[i])

Bug's Life, A (1998) pixar
Heathers (1989) highschool
Ferris Bueller's Day Off (1986) highschool
Grease 2 (1982) highschool
Grease (1978) highschool
Dead Poets Society (1989) highschool highschool
Stand and Deliver (1988) highschool
Fast Times at Ridgemont High (1982) highschool
Napoleon Dynamite (2004) highschool
Never Been Kissed (1999) highschool
